In [14]:
### Importing all the neccessary packages ###
import numpy as np

import xarray as xr
import matplotlib.pyplot as plt
import pandas as pd
import cartopy.crs as ccrs
import cartopy
from matplotlib.offsetbox import AnchoredText
import cartopy.feature as cfeature
import scipy.fft as sf
from scipy import signal
from scipy.stats import circmean
from scipy import optimize
from mpl_toolkits.axes_grid1 import make_axes_locatable
import time
import fsspec
#import intake_esm as intake
#import pandas as pd
from importlib import reload
import cartopy.io.shapereader as shapereader


In [15]:
import polars as pl

In [16]:
import seasonality.seasonalityfunctions as sf
import seasonality.onset_demise as od

In [17]:
reload(od)
reload(sf)

<module 'seasonality.seasonalityfunctions' from '/Users/gbromley/code/PrecipitationSeasonality/seasonality/seasonalityfunctions.py'>

In [18]:
### Allows us to use dask to speed up some calculations ###
from dask.distributed import Client
client = Client()
client


Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:8787/status,
Dashboard: http://127.0.0.1:8787/status,Workers: 4
Total threads: 8,Total memory: 16.00 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:61042,Workers: 4
Dashboard: http://127.0.0.1:8787/status,Total threads: 8
Started: Just now,Total memory: 16.00 GiB
Comm: tcp://127.0.0.1:61055,Total threads: 2
Dashboard: http://127.0.0.1:61058/status,Memory: 4.00 GiB
Nanny: tcp://127.0.0.1:61045,


In [19]:
hist_pr = xr.open_dataset('CESM2_CMIP6_historical.nc')
hist_pr = hist_pr.mean(dim='member')
hist_pr['time'] = pd.date_range('1850-01-01', freq='D', periods=60228)
hist_pr = hist_pr['pr'] * 86400

In [20]:
future_pr = xr.open_dataset('pr_day_CESM2_ssp585_r4i1p1f1_gn_20500101-20991231_v20200528.nc')
future_pr['time'] = pd.date_range('2050-01-01', freq='D', periods=18250)
future_pr = future_pr['pr'] * 86400

In [21]:
### Takes 35 seconds

hist_start = od.B17_analysis_start(sf.calc_annual_cycle(hist_pr))
future_start = od.B17_analysis_start(sf.calc_annual_cycle(future_pr))

In [22]:
def plot_spatial_data(dataarray, projection=ccrs.PlateCarree(), title='Spatial Data Plot', var='data_to_plot'):
    """
    Plots a spatial figure of a variable from an xarray DataArray.

    :param dataarray: xarray DataArray containing the geospatial data to be plotted.
    :param projection: Cartopy CRS projection. Defaults to PlateCarree.
    :param title: Title of the plot.
    """
    
    states_provinces = cfeature.NaturalEarthFeature(
    category='cultural',
    name='admin_1_states_provinces_lines',
    scale='10m')
    map_proj = ccrs.LambertConformal(central_longitude=-95, central_latitude=45)
    #cmap = mpl.cm.RdBu_r


    fig, ax = plt.subplots(1, 1, figsize=(10, 13), dpi=600,  subplot_kw={'projection': projection})
    p = dataarray.plot(ax=ax,transform=ccrs.PlateCarree(), add_colorbar=False, cmap='twilight')


    ### Setting 1st plot parameters ###
    ax.coastlines(edgecolor='black')
    ax.add_feature(cartopy.feature.BORDERS, edgecolor='black')
    ax.add_feature(cfeature.STATES, edgecolor='black')
    ax.add_feature(cfeature.LAKES, alpha=0.5, edgecolor='blue')
    ax.add_feature(cfeature.RIVERS, color='blue')
    #ax1.set_xticks(np.arange(-180,181, 40))
    #ax1.set_yticks(np.arange(-90,91,15))
    drainage = shapereader.Reader('/Users/gbromley/Downloads/major_basins_of_the_world_0_0_0/Major_Basins_of_the_World.shp')
    for feature in drainage.records():
        geometry = feature.geometry
        ax.add_geometries([geometry], ccrs.PlateCarree(), facecolor='none', edgecolor='grey', )
    #at = AnchoredText("a",
    #                    loc='upper left', prop=dict(size=8), frameon=True,)
    #at.patch.set_boxstyle("round,pad=0.,rounding_size=0.2")
    #ax1.add_artist(at)
    divider = make_axes_locatable(ax)
    cax = divider.append_axes("right", size="5%", pad=0.1, axes_class=plt.Axes)
    plt.colorbar(p, cax=cax, label=var)
    
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    gl = ax.gridlines(draw_labels=True, dms=True, x_inline=False, y_inline=False)
    gl.left_labels = True
    gl.right_labels = False
    gl.top_labels = False
    gl.bottom_labels = True

    # Add a title
    ax.set_title(title, loc='center')

    # Show the plot
    plt.show()

In [23]:
def checkData(data):
    return type(data)

In [24]:
def si_to_mm(data):
    data = data * 86400
    return data
    

In [25]:
def calc_anom(data, var='pr'):
    
    ### Create The annual daily precip anomalies ###
    annual_means = data.groupby('time.year').mean(dim='time')


    anomalies = data.groupby('time.year') - annual_means
    #anomalies = anomalies.where(anomalies['time'] != )

    #climDailyMeanAnomaly = annual_precip_cycle - annual_mean_precip
    
    #remove leap days
    #mask = (anomalies['time.month'] != 2) & (anomalies['time.day'] != 29)
    #anomalies = anomalies.where(mask)
    
    return  anomalies

In [26]:
### Peru Domain ###
min_lon = -83+360
min_lat = -20.0
max_lon = -67+360
max_lat = 0.0

In [27]:
#future_pr = si_to_mm(future_pr)
#hist_pr = si_to_mm(hist_pr)

future_anom = calc_anom(future_pr)
hist_anom = calc_anom(hist_pr)

In [15]:
years = hist_anom['time.year']
days = hist_anom['time.dayofyear']

In [ ]:
data = test_anom
years = data.time.values.astype('datetime64[Y]')
days = data.time.values.astype('datetime64[D]') - years
days = days.astype('int')

data = data.values
#if len(days) != len(years):
    #raise ValueError('Length of days and years must be the same.')
    
# Want to make sure we get all the input years before any trimming
unique_years = np.unique(years)

onsetDOY = np.empty((len(unique_years)))
onsetDOY[:] = np.nan



# Day of year is missing integer 60 which is February 29th.
# Need to add zero because a tuple is returned from np.where
start_index = np.where(days == 258.0)[0]





### looping through start dates ###
for start_day in start_index:
    
    analysis_begin = start_day
    analysis_end = start_day + 365
    

    
    if (analysis_end > len(data)):
        analysis_end = len(data)
    
    analysis_days = days[analysis_begin:analysis_end]
    analysis_years = years[analysis_begin:analysis_end] 
    
    cumsum_data = sf.cumul_anom(data, analysis_begin, analysis_end)
    
    # this returns the index of the data not the day
    
    onset_index = np.argmin(cumsum_data)
    onset_day = analysis_days[onset_index]
    onset_year = analysis_years[onset_index]
    #print(onset_index)
    where_to_place = np.argwhere(unique_years == onset_year)[0][0]
    onsetDOY[where_to_place] = onset_day
    
    


In [ ]:
xr.apply_ufunc(
    test_onset,
    data,
    data.time,
    startWet,
    input_core_dims=[["time"],["time"],[]],
    exclude_dims=set(["time"]),
    output_core_dims=[["year"]],
    vectorize=True,
    dask = 'parallelized',
    #output_dtypes = 'datetime64[D]',
    #output_sizes={"data_jday": 71},
)

In [30]:
def _onset_LM01(data, startWet):
    xr.apply_ufunc(
    test_onset,
    data,
    data.time,
    startWet,
    input_core_dims=[["time"],["time"],[]],
    exclude_dims=set(["time"]),
    output_core_dims=[["year"]],
    vectorize=True,
    dask = 'parallelized',
    #output_dtypes = 'datetime64[D]',
    #output_sizes={"data_jday": 71},
)

In [31]:
test_anom = hist_anom.isel(lat=1, lon=1)
test_start = hist_start.isel(lat=1, lon=1)

In [74]:
pd.to_datetime(test_anom.time)

DatetimeIndex(['1850-01-01', '1850-01-02', '1850-01-03', '1850-01-04',
               '1850-01-05', '1850-01-06', '1850-01-07', '1850-01-08',
               '1850-01-09', '1850-01-10',
               ...
               '2014-11-15', '2014-11-16', '2014-11-17', '2014-11-18',
               '2014-11-19', '2014-11-20', '2014-11-21', '2014-11-22',
               '2014-11-23', '2014-11-24'],
              dtype='datetime64[ns]', length=60228, freq=None)

In [42]:
pd.date_range('2001-01-01', '2001-05-05', freq='D')

DatetimeIndex(['2001-01-01', '2001-01-02', '2001-01-03', '2001-01-04',
               '2001-01-05', '2001-01-06', '2001-01-07', '2001-01-08',
               '2001-01-09', '2001-01-10',
               ...
               '2001-04-26', '2001-04-27', '2001-04-28', '2001-04-29',
               '2001-04-30', '2001-05-01', '2001-05-02', '2001-05-03',
               '2001-05-04', '2001-05-05'],
              dtype='datetime64[ns]', length=125, freq='D')

In [77]:
np.array(pd.to_datetime(['2024-01-01','1978-5-13','2024-2-29']).dayofyear)

array([  1, 133,  60], dtype=int32)

In [79]:
np.unique(pd.date_range('2000-01-01', '2001-05-05', freq='D').year)

array([2000, 2001], dtype=int32)

In [90]:
days = np.array(pd.date_range('2000-01-01', '2001-05-05', freq='D').day)

In [91]:
days

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,  1,  2,  3,
        4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20,
       21, 22, 23, 24, 25, 26, 27, 28, 29,  1,  2,  3,  4,  5,  6,  7,  8,
        9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25,
       26, 27, 28, 29, 30, 31,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11,
       12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28,
       29, 30,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15,
       16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31,  1,
        2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18,
       19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30,  1,  2,  3,  4,  5,
        6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
       23, 24, 25, 26, 27, 28, 29, 30, 31,  1,  2,  3,  4,  5,  6,  7,  8,
        9, 10, 11, 12, 13

In [89]:
np.squeeze(np.where(days == 278))

array(277)

In [93]:
def test_onset(data,time, startWet):
    
    time = pd.to_datetime(time)
    years = np.array(time.year)
    days = np.array(time.dayofyear)
    #days = days.astype('int')
    
    #if len(days) != len(years):
        #raise ValueError('Length of days and years must be the same.')
        
    # Want to make sure we get all the input years before any trimming
    unique_years = np.unique(years)
    
    onsetDOY = np.empty((len(unique_years)))
    onsetDOY[:] = np.nan
    
    
    
    # Day of year is missing integer 60 which is February 29th.
    # Need to add zero because a tuple is returned from np.where
    start_index = np.squeeze(np.where(days == startWet))
    
    
    

    print(days)
    ### looping through start dates ###
    for start_day in start_index:
        
        analysis_begin = start_day
        analysis_end = start_day + 365
        

        
        if (analysis_end > len(data)):
            analysis_end = len(data)
        
        analysis_days = days[analysis_begin:analysis_end]
        analysis_years = years[analysis_begin:analysis_end] 
        
        cumsum_data = sf.cumul_anom(data, analysis_begin, analysis_end)
        
        # this returns the index of the data not the day
        
        onset_index = np.argmin(cumsum_data)
        onset_day = analysis_days[onset_index]
        onset_year = analysis_years[onset_index]
        #print(onset_index)
        where_to_place = np.argwhere(unique_years == onset_year)[0][0]
        onsetDOY[where_to_place] = onset_day
    #print(onsetDOY)
    return onsetDOY
    
    


In [94]:
test_onset(test_anom.values, test_anom.time, test_start.values)

Index([  1,   2,   3,   4,   5,   6,   7,   8,   9,  10,
       ...
       319, 320, 321, 322, 323, 324, 325, 326, 327, 328],
      dtype='int32', length=60228)


array([338., 356., 329., 359., 352., 348.,  nan, 350., 324.,  nan, 349.,
       361., 340., 340., 353., 334., 342.,  nan, 349., 365., 342., 341.,
       348., 338., 335.,  nan, 318., 348., 327., 335., 336., 342., 330.,
        nan, 363., 320., 330., 356., 328., 320., 318., 353., 336., 333.,
       363., 327., 331., 341., 357., 346., 360., 344., 319., 330., 346.,
       327., 365., 325., 331., 347., 346., 338., 335., 360., 319., 332.,
       342., 326., 329., 321., 342., 322., 347., 339., 362., 352., 350.,
       315., 314., 296., 313., 362., 337., 363., 334., 325., 359., 336.,
       338.,  nan, 335., 324.,  nan, 319., 333., 362., 339., 327., 358.,
        nan, 321., 285., 337., 335., 325., 333., 323., 328., 337., 319.,
       311.,  nan,  84., 166., 327., 315., 359., 325., 320., 318., 324.,
       314., 315., 329., 296., 337., 302., 321., 341., 349., 326., 326.,
       343., 321., 318.,  nan, 320., 325., 341., 323.,  nan, 359., 315.,
       310., 340., 299., 330.,  nan, 322., 279., 32

In [13]:
test_anom.time

NameError: name 'test_anom' is not defined

In [41]:
%%timeit
pd.date_range('1-1-2029', '12-31-2029').day_of_year

96.4 µs ± 1.1 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [10]:
from datetime import date

In [37]:
pl.from_dataframe(hist_pr.to_dataframe())

pr
f64
0.003954
0.012041
0.009864
0.011923
0.009319
0.005268
0.002213
0.049869
0.192289


In [28]:
pl.date(2023,1,1)

<Expr ['2023.dt.datetime([1, 1, 0, 0, …'] at 0x1420FEB90>